In [1]:
from transformers import BertTokenizer, BertForSequenceClassification, pipeline
from pprint import pprint

c:\Users\josea\Desktop\backend-projects\music-mood-backend\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_path = 'fine_tuned_model'

model = BertForSequenceClassification.from_pretrained(model_path)
tokenizer = BertTokenizer.from_pretrained(model_path)
classifier = pipeline('text-classification', model=model, tokenizer=tokenizer, return_all_scores=True)

Device set to use cuda:0
c:\Users\josea\Desktop\backend-projects\music-mood-backend\.venv\Lib\site-packages\transformers\pipelines\text_classification.py:111: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [3]:
threshold = 0.3

inputs = [
	'Eu te amo',
	'Eu acho que você é uma ótima pessoa',
	'Eu odeio aquele cara',
	]

output = classifier(inputs)

predictions = []

for prediction in output:
	predictions.append(list(x for x in prediction if x['score']>= threshold))

pprint(predictions)

[[{'label': 'amor', 'score': 0.9788758754730225}],
 [{'label': 'admiração', 'score': 0.9857978224754333}],
 [{'label': 'raiva', 'score': 0.7176174521446228}]]


In [5]:
%pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [8]:
%pip install openpyxl


   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpy

In [6]:
import pandas as pd

In [9]:
frases_emocoes = [
    "Ansioso para amanhã, me recomenda algo relaxante para ouvir antes de dormir?",
    "Tô meio triste, queria ouvir algo mais calmo, tipo uma balada melancólica.",
    "Estou muito feliz hoje, quero uma música animada para comemorar.",
    "Acabei de passar numa prova difícil, preciso de uma trilha sonora para comemorar!",
    "Tá chovendo e eu só queria uma playlist tranquila para ficar em casa.",
    "Terminei um relacionamento, não quero nada muito animado agora.",
    "Tá chovendo e eu só queria uma playlist tranquila para ficar em casa.",
    "Estou com muita raiva, preciso de um som pesado para descarregar.",
    "Bateu uma saudade enorme, queria ouvir músicas que me lembrem minha infância.",
    "Tô nostálgico hoje, me recomenda umas músicas antigas dos anos 90?",
    "Ansioso e sem foco, preciso de algo instrumental para me acalmar.",
    "Estou eufórico, quero músicas bem agitadas para manter a energia lá em cima!",
    "Tô entediado, manda qualquer coisa diferente para eu descobrir artistas novos.",
    "Preciso de músicas inspiradoras para me motivar a começar a semana.",
    "Estou apaixonado, quero músicas românticas para entrar no clima.",
    "Sinto um vazio, quero algo profundo e reflexivo."
]

resultados = []

for frase in frases_emocoes:
    predicoes = classifier(frase)[0]  # retorna lista de dicts [{'label':..., 'score':...}, ...]
    linha = {"Frase": frase}
    for p in predicoes:
        linha[p["label"]] = p["score"]  # adiciona cada emoção como coluna
    resultados.append(linha)

# Cria DataFrame
df = pd.DataFrame(resultados)

# Salva em Excel
df.to_excel("resultados_emocoes.xlsx", index=False)

print("Arquivo 'resultados_emocoes.xlsx' gerado com sucesso!")

Arquivo 'resultados_emocoes.xlsx' gerado com sucesso!


In [ ]:
from openpyxl import load_workbook
from openpyxl.styles import PatternFill

In [ ]:



resultados = []
for frase in frases_emocoes:
    predicoes = classifier(frase)[0]  
    linha = {"Frase": frase}
    for p in predicoes:
        linha[p["label"]] = p["score"]
    resultados.append(linha)

# === SALVAR EM EXCEL ===
arquivo_excel = "resultados_emocoes2.xlsx"
df = pd.DataFrame(resultados)
df.to_excel(arquivo_excel, index=False)

# === DESTACAR EMOÇÃO DOMINANTE ===
wb = load_workbook(arquivo_excel)
ws = wb.active

# Descobre índices das colunas
headers = [cell.value for cell in ws[1]]
col_frase = headers.index("Frase") + 1
col_emocoes = list(range(col_frase + 1, len(headers) + 1))

# Estilo para destaque (amarelo)
fill = PatternFill(start_color="FFF59D", end_color="FFF59D", fill_type="solid")

# Para cada linha (ignorando cabeçalho)
for row in range(2, ws.max_row + 1):
    valores = [ws.cell(row=row, column=col).value for col in col_emocoes]
    if not valores:
        continue
    max_val = max(valores)
    for col in col_emocoes:
        if ws.cell(row=row, column=col).value == max_val:
            ws.cell(row=row, column=col).fill = fill

wb.save(arquivo_excel)

print(f"Arquivo '{arquivo_excel}' gerado com emoções destacadas!")
